# How to use parameters and dynamic parameters

Parameters and dynamic parameters are features of Dapper that let you pass values and types to SQL queries securely and efficiently. They can protect your code from SQL injection attacks, enhance its performance, and make it easier to read and maintain.

To use parameters with Dapper, you need to match the names of the parameters in your query to the names of the properties in your C# entity and pass the entity as the parameters. You can follow these steps:
* Create a class that represents the data model.
* Create a connection object that connects to the database. 
* Use the <code>Query</code> or <code>QueryAsync</code> method to execute a SQL query or a stored procedure that returns a list of objects. You can pass a parameter object that contains the values for the parameters in the query or the procedure. For example:


In [1]:
// Connection string
#load "AppSettings.cs"

In [2]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:Dapper"

Installed Packages Dapper, 2.1.28 Microsoft.Data.SqlClient, 5.1.4

In [3]:
using Microsoft.Data.SqlClient;
using System.Data;
using System.Data.Common;
using Dapper;

In [4]:
public class Location
{
    public short LocationID { get; set; }
    public string Name { get; set; }
    public decimal CostRate { get; set; }
    public decimal Availability { get; set; }
    public DateTime ModifiedDate { get; set; }

    public void Display()
    {       
        Console.WriteLine($"LocationID: {LocationID}");
        Console.WriteLine($"Name: {Name}");
        Console.WriteLine($"CostRate: {CostRate}");
        Console.WriteLine($"Availability: {Availability}");
        Console.WriteLine($"ModifiedDate: {ModifiedDate}");
    }
}

public class Employee
{
    public int RecursionLevel { get; set; }
    public string OrganizationNode { get; set; }
    public string ManagerFirstName { get; set; }
    public string ManagerLastName { get; set; }
    public int BusinessEntityID { get; set; }
    public string FirstName { get; set; }
    public string LastName { get; set; }

    public void Display()
    {       
        Console.WriteLine($"RecursionLevel: {RecursionLevel}");
        Console.WriteLine($"OrganizationNode: {OrganizationNode}");
        Console.WriteLine($"ManagerFirstName: {ManagerFirstName}");
        Console.WriteLine($"ManagerLastName: {ManagerLastName}");
        Console.WriteLine($"BusinessEntityID: {BusinessEntityID}");
        Console.WriteLine($"BusinessEntityID: {FirstName}");
        Console.WriteLine($"BusinessEntityID: {LastName}");        
    }
}

In [5]:
using (var connection = new SqlConnection(connectionString))
{    
    // query with a single parameter
    var query = "SELECT * FROM Production.Location WHERE LocationID = @LocationID";
    var parameter = new { LocationID = 1 };
    var result = connection.Query<Location>(query, parameter);
    Console.WriteLine("\nQuery with a single parameter:\n");
    foreach(var location in  result)
    {
        location.Display();
    }
}

using (var connection = new SqlConnection(connectionString))
{    
    // query with multiple parameters
    var query = "SELECT * FROM Production.Location WHERE CostRate BETWEEN @MinCostRate AND @MaxCostRate";
    var parameters = new { MinCostRate = 10, MaxCostRate = 20 };
    var result = connection.Query<Location>(query, parameters);
    Console.WriteLine("\nQuery with multiple parameters:\n");
    foreach(var location in  result)
    {
        location.Display();
    }
}

using (var connection = new SqlConnection(connectionString))
{        
    // stored procedure with a single parameter
    var procedure = "uspGetManagerEmployees";
    var parameter = new { BusinessEntityID = 5 };
    var result = connection.Query<Employee>(procedure, parameter, commandType: CommandType.StoredProcedure);
    Console.WriteLine("\nStored procedure with a single parameter:\n");
    result.Display();   
}


Query with a single parameter:

LocationID: 1
Name: Tool Crib
CostRate: 0.0000
Availability: 0.00
ModifiedDate: 4/30/2008 12:00:00 AM

Query with multiple parameters:

LocationID: 30
Name: Debur and Polish
CostRate: 14.5000
Availability: 120.00
ModifiedDate: 4/30/2008 12:00:00 AM
LocationID: 40
Name: Paint
CostRate: 15.7500
Availability: 120.00
ModifiedDate: 4/30/2008 12:00:00 AM
LocationID: 45
Name: Specialized Paint
CostRate: 18.0000
Availability: 80.00
ModifiedDate: 4/30/2008 12:00:00 AM
LocationID: 50
Name: Subassembly
CostRate: 12.2500
Availability: 120.00
ModifiedDate: 4/30/2008 12:00:00 AM
LocationID: 60
Name: Final Assembly
CostRate: 12.2500
Availability: 120.00
ModifiedDate: 4/30/2008 12:00:00 AM

Stored procedure with a single parameter:



index value 0 Submission#4+Employee RecursionLevel 0 OrganizationNode /1/1/2/ ManagerFirstName Roberto ManagerLastName Tamburello BusinessEntityID 5 FirstName Gail LastName Erickson

* You can use the <code>DynamicParameters</code> class to create a dynamic parameter object that can handle various types of parameters, such as input, output, return value, or table-valued parameters. Here is a simple example of using the <code>DynamicParameters</code> class with C# and Dapper.

1. First, let’s use the existing stored procedure named <code>uspGetBillOfMaterials</code> that takes two parameters: <code>@StartProductID</code> and <code>@CheckDate</code>. The stored procedure returns the list of components and subassemblies needed to build a product on a given date.
2. To call this stored procedure using C# and Dapper, we need to create a dynamic parameter object and add the values for the parameters. We also need to specify the data type and the direction of each parameter. 


In [6]:
public class BillOfMaterials
{    
    public int ProductAssemblyID { get; set; }
    public int ComponentID { get; set; }
    public string ComponentDesc { get; set; }
    public decimal PerAssemblyQty { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public int BOMLevel { get; set; }
    public int RecursionLevel { get; set; }  
  
    public void Display()
    {       
        Console.WriteLine($"ProductAssemblyID: {ProductAssemblyID}");
        Console.WriteLine($"ComponentID: {ComponentID}");
        Console.WriteLine($"ComponentDesc: {ComponentDesc}");
        Console.WriteLine($"PerAssemblyQty: {PerAssemblyQty}");
        Console.WriteLine($"StandardCost: {StandardCost}");
        Console.WriteLine($"ListPrice: {ListPrice}");
        Console.WriteLine($"BOMLevel: {BOMLevel}");
        Console.WriteLine($"RecursionLevel: {RecursionLevel}");
    }
}

using (var connection = new SqlConnection(connectionString))
{
    // create a dynamic parameter object
    var dynamicParameter = new DynamicParameters();

    // add the values for the stored procedure parameters
    dynamicParameter.Add("@StartProductID", 726, DbType.Int32, ParameterDirection.Input);
    dynamicParameter.Add("@CheckDate", new DateTime(2010, 5, 26), DbType.DateTime, ParameterDirection.Input);

    // execute the stored procedure and get a list of BillOfMaterials objects
    var result = connection.Query<BillOfMaterials>("uspGetBillOfMaterials", dynamicParameter, commandType: CommandType.StoredProcedure);
    result.Display();
}

index value 0 Submission#6+BillOfMaterials ProductAssemblyID 726 ComponentID 493 ComponentDesc Paint - Red PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 2 RecursionLevel 0 1 Submission#6+BillOfMaterials ProductAssemblyID 726 ComponentID 532 ComponentDesc Seat Stays PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 2 RecursionLevel 0 2 Submission#6+BillOfMaterials ProductAssemblyID 726 ComponentID 533 ComponentDesc Seat Tube PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 2 RecursionLevel 0 3 Submission#6+BillOfMaterials ProductAssemblyID 726 ComponentID 534 ComponentDesc Top Tube PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 2 RecursionLevel 0 4 Submission#6+BillOfMaterials ProductAssemblyID 533 ComponentID 478 ComponentDesc Metal Bar 2 PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 3 RecursionLevel 1 5 Submission#6+BillOfMaterials ProductAssemblyID 534 ComponentID 482 ComponentDesc Metal Sheet 2 PerAssemblyQty 0 StandardCost 0.0000 ListPrice 0.0000 BOMLevel 3 RecursionLevel 1